In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 9f14b60
molecule = {'h2o': 'atmpro', 'co2': 0.0004, 'n2o': 3.2e-07}
band = [3]
atmpro = mls
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band3a
  atmpro = mls
  band = 3a
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 0
  molecule = h2o
  ng_adju = [0, 0]
  ng_refs = [2, 6]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(10, 250), (600, 250)]
  tsfc = 294
  vmax = 620
  vmin = 540
  w_diffuse = [(1.9, 1.7), (1.4, 1.4, 1.4, 1.55, 1.6, 1.66)]
  wgt = [(0.7, 0.7), (0.7, 0.5, 0.5, 0.5, 0.5, 0.5)]
o co2 band3a
  atmpro = mls
  band = 3a
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.375563e-24
  molecule = co2
  ng_adju = [0, 0, 0]
  ng_refs = [3, 2, 4]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (10, 250), (500, 250)]
  tsfc = 294
  vmax = 620
  vmin = 540
  w_diffuse = [(1.6, 1.6, 1.7), (1.75, 1.75), (1.55, 1.55, 1.6, 1.85)]
  wgt = [(0.7, 0.3, 0.7), (0

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0  0.393184                  NaN       NaN
0.000312      NaN       NaN                  1.0 -1.285852
0.000750      2.0  0.392150                  2.0  0.132404
0.001052      3.0  0.403698                  3.0  0.297371
0.001476      4.0  0.414500                  4.0  0.469899
0.002070      5.0  0.423268                  5.0  0.647620
0.002904      6.0  0.428759                  6.0  0.826845
0.004074      7.0  0.430491                  7.0  1.004970
0.005714      8.0  0.427059                  8.0  1.179504
0.008015      9.0  0.417199                  9.0  1.347720
0.011243     10.0  0.399614                 10.0  1.507313
0.015771     11.0  0.372704                 11.0  1.655994
0.022122     12.0  0.334923                 12.0  1.789733
0.031031     13.0  0.284540                 13.0  1.904908
0.043528     14.0  0.219733                 14.0  1.999412
0.061057     15.0  0.171438                 15.0  2.100359
0.085645     16.0  0.194250                 16.0  2.271167
0.120136     17.0  0.264847                 17.0  2.504513
0.168516     18.0  0.322976                 18.0  2.719230
0.236378     19.0  0.372871                 19.0  2.875972
0.331549     20.0  0.429965                 20.0  2.977813
0.465100     21.0  0.512670                 21.0  3.064141
0.652400     22.0  0.652597                 22.0  3.185447
0.915100     23.0  0.811154                 23.0  3.236423
1.283650     24.0  0.780785                 24.0  3.016818
1.800600     25.0  0.612281                 25.0  2.594347
2.525700     26.0  0.478381                 26.0  2.159360
3.542800     27.0  0.378498                 27.0  1.779270
4.969550     28.0  0.300502                 28.0  1.458376
6.970850     29.0  0.241145                 29.0  1.188576
9.778100     30.0  0.194278                 30.0  0.949142
13.715850    31.0  0.152132                 31.0  0.731363
19.239350    32.0  0.113647                 32.0  0.538478
26.987250    33.0  0.086951                 33.0  0.381211
37.855300    34.0  0.070629                 34.0  0.260664
53.100050    35.0  0.054172                 35.0  0.166450
73.887500    36.0  0.036771                 36.0  0.096189
97.662500    37.0  0.026113                 37.0  0.085215
121.437500   38.0  0.020571                 38.0  0.070388
145.212500   39.0  0.010851                 39.0  0.052331
168.987500   40.0  0.000545                 40.0  0.037652
192.762500   41.0  0.004399                 41.0  0.037625
216.537500   42.0  0.025884                 42.0  0.052262
240.312500   43.0  0.051967                 43.0  0.078709
264.087500   44.0  0.080338                 44.0  0.111245
287.862500   45.0  0.109029                 45.0  0.143516
311.637500   46.0  0.127294                 46.0  0.175415
335.412500   47.0  0.143269                 47.0  0.200792
359.187500   48.0  0.160523                 48.0  0.216278
382.962500   49.0  0.178517                 49.0  0.222105
406.737500   50.0  0.196831                 50.0  0.219562
430.512500   51.0  0.215039                 51.0  0.210048
454.287500   52.0  0.230139                 52.0  0.195828
478.062500   53.0  0.243821                 53.0  0.178169
501.837500   54.0  0.257265                 54.0  0.158783
525.612500   55.0  0.270299                 55.0  0.139797
549.387500   56.0  0.282760                 56.0  0.122977
573.162500   57.0  0.294696                 57.0  0.109715
596.937500   58.0  0.314987                 58.0  0.097873
620.712500   59.0  0.332144                 59.0  0.088091
644.487500   60.0  0.343727                 60.0  0.079460
668.262500   61.0  0.349933                 61.0  0.071460
692.037500   62.0  0.350810                 62.0  0.064241
715.812500   63.0  0.347014                 63.0  0.058204
739.587500   64.0  0.339288                 64.0  0.053983
76

# Fluxes by Layer

CRD                           CLIRAD             \
                         flug       fldg      fnetg       flug       fldg   
pressure     level                                                          
0.000000e+00 1     -24.866066   0.000000 -24.866066        NaN        NaN   
1.000000e-08 1            NaN        NaN        NaN -15.541766   0.000002   
6.244000e-04 2     -24.866085   0.000048 -24.866037 -15.541899   0.000039   
8.759000e-04 3     -24.866092   0.000067 -24.866026 -15.541947   0.000092   
1.228600e-03 4     -24.866102   0.000094 -24.866009 -15.542007   0.000164   
1.723400e-03 5     -24.866117   0.000132 -24.865984 -15.542082   0.000267   
2.417400e-03 6     -24.866136   0.000187 -24.865950 -15.542174   0.000412   
3.390900e-03 7     -24.866163   0.000263 -24.865900 -15.542284   0.000617   
4.756500e-03 8     -24.866201   0.000370 -24.865830 -15.542412   0.000908   
6.672000e-03 9     -24.866253   0.000520 -24.865734 -15.542557   0.001321   
9.358900e-03 10    -24.866326   0.000725 -24.865601 -15.542713   0.001905   
1.312780e-02 11    -24.866428   0.001005 -24.865422 -15.542866   0.002732   
1.841450e-02 12    -24.866572   0.001383 -24.865189 -15.542992   0.003895   
2.583020e-02 13    -24.866777   0.001882 -24.864895 -15.543049   0.005524   
3.623230e-02 14    -24.867074   0.002530 -24.864544 -15.542963   0.007786   
5.082340e-02 15    -24.867520   0.003356 -24.864164 -15.542614   0.010893   
7.129060e-02 16    -24.868193   0.004444 -24.863749 -15.541778   0.015150   
1.000000e-01 17    -24.869152   0.006064 -24.863088 -15.540016   0.021113   
1.402710e-01 18    -24.870424   0.008600 -24.861825 -15.536629   0.029674   
1.967600e-01 19    -24.872053   0.012389 -24.859663 -15.530706   0.041948   
2.759970e-01 20    -24.874044   0.017880 -24.856164 -15.520975   0.059215   
3.871000e-01 21    -24.876287   0.025782 -24.850505 -15.505684   0.083119   
5.431000e-01 22    -24.878360   0.037329 -24.841031 -15.482391   0.116455   
7.617000e-01 23    -24.878715   0.054583 -24.824132 -15.447886   0.164444   
1.068500e+00 24    -24.873968   0.079315 -24.794653 -15.400414   0.234605   
1.498800e+00 25    -24.863469   0.108615 -24.754855 -15.344319   0.332300   
2.102400e+00 26    -24.850190   0.139114 -24.711076 -15.285932   0.459430   
2.949000e+00 27    -24.835224   0.172123 -24.663101 -15.228293   0.618366   
4.136600e+00 28    -24.818919   0.209065 -24.609854 -15.173276   0.813681   
5.802500e+00 29    -24.801679   0.251125 -24.550554 -15.123608   1.051836   
8.139200e+00 30    -24.783665   0.299860 -24.483805 -15.082864   1.340123   
1.141700e+01 31    -24.765059   0.356688 -24.408371 -15.057346   1.683174   
1.601470e+01 32    -24.747524   0.422008 -24.325515 -15.056687   2.080879   
2.246400e+01 33    -24.735327   0.496634 -24.238693 -15.093225   2.528839   
3.151050e+01 34    -24.732014   0.586499 -24.145514 -15.180878   3.025047   
4.420010e+01 35    -24.739676   0.700330 -24.039346 -15.340343   3.576376   
6.200000e+01 36    -24.767654   0.842531 -23.925123 -15.610221   4.197254   
8.577500e+01 37    -24.833649   1.012084 -23.821565 -16.038842   4.896802   
1.095500e+02 38    -24.923255   1.175232 -23.748023 -16.549432   5.647408   
1.333250e+02 39    -25.030564   1.340476 -23.690088 -17.127711   6.423942   
1.571000e+02 40    -25.162603   1.503075 -23.659529 -17.754151   7.197777   
1.808750e+02 41    -25.329943   1.671948 -23.657995 -18.403667   7.953344   
2.046500e+02 42    -25.526904   1.881297 -23.645607 -19.052285   8.707937   
2.284250e+02 43    -25.740613   2.167904 -23.572709 -19.700042   9.502897   
2.522000e+02 44    -25.968923   2.542569 -23.426354 -20.348183  10.372729   
2.759750e+02 45    -26.214597   3.014501 -23.200097 -21.002828  11.340708   
2.997500e+02 46    -26.481855   3.588819 -22.893036 -21.673456  12.415565   
3.235250e+02 47    -26.764260   4.229724 -22.534536 -22.343814  13.579997   
3.473000e+02 48    -27.058341   4.927296 -22.131046 -23.005153  14.806888   
3.710750e+02 49    -27

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')